In [ ]:
from collections import Counter
from matplotlib.lines import Line2D
import numpy as np
import pandas as pd
import json
import pickle
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from copy import deepcopy
import pandas as pd
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import random

In [ ]:
df = 'your_file.tsv' #Dataset .tsv file

df = pd.read_csv(df,sep='\t')

In [ ]:
n_classes = df[' class'].nunique()

contagem_por_classe = df[' class'].value_counts()

porcentagens_por_classe = (contagem_por_classe / len(df[' class']))

n_classes = df[' class'].nunique()
print(f'There are {n_classes} classes')

per_por_class = []  

for classe, porcentagem in porcentagens_por_classe.items():
    print(f'% {classe} is: {porcentagem:.2f}')
    per_por_class.append(porcentagem)  

print(per_por_class)

In [ ]:
numeric_columns = []
integer_columns = []

for col in df.columns:
    if df[col].dtype == 'float64':
        numeric_columns.append(col)
    elif df[col].dtype == 'int64':
        integer_columns.append(col)
print('float columns are:', numeric_columns)
print('int columns are:', integer_columns)

In [ ]:
scaler = StandardScaler()
df[numeric_columns] = scaler.fit_transform(df[numeric_columns])

## Obtenção dos Triclusters

In [ ]:
nome_arquivo = "your_triclusters_output.txt"  #The triclusters solution obtained from TCTriCluster .txt

with open(nome_arquivo, 'r') as arquivo:
    tctri = arquivo.read()

In [ ]:
linhas = tctri.strip().split('\n')

tctriclass = []

def calcular_classe_majoritaria(linha):
    classes = [int(c) for c in linha.strip('[]').split(', ')]
    total = len(classes)
    classe_majoritária = max(set(classes), key=classes.count)
    percentagem = (classes.count(classe_majoritária) / total)
    return [classe_majoritária, percentagem, total]

for linha in linhas:
    resultado = calcular_classe_majoritaria(linha)
    tctriclass.append(resultado)

tctriclass

In [ ]:
contagem_por_classe = [sum(1 for item in tctriclass if item[0] == classe) for classe in range(n_classes)]

porcentagens_por_classe = [contagem / len(tctriclass) * 100 for contagem in contagem_por_classe]

for classe, porcentagem in enumerate(porcentagens_por_classe):
    print(f'% of triclusters with the majority class {classe} is: {porcentagem:.2f}%')

In [ ]:
tctri_classes, tctri_ratios, tctri_elements = zip(*tctriclass)

bins = [i for i in range(0, 61, 10)]

elements_binned = np.histogram(tctri_elements, bins=bins)[0]

bin_labels = [f'{bins[i]}-{bins[i+1]-1}' for i in range(len(bins)-1)]

plt.bar(bin_labels, elements_binned, color='r', edgecolor='black')

plt.xlabel('Number of elements')
plt.ylabel('Frequency')

plt.show()

### Filtragem dos Triclusters com o poder discriminativo threshold

In [ ]:
def selecionar_triclusters_por_razao(tctriclass, min_razao):
    selected_indices = []

    for idx, (cls, ratio, elements) in enumerate(tctriclass):
        if ratio >= min_razao:
            selected_indices.append([idx,ratio])

    return selected_indices

min_razao = 0.6 #DS threshold parameter
selected_indices = selecionar_triclusters_por_razao(tctriclass, min_razao)
print("There were selected", len(selected_indices), "triclusters.")
selected_indices_modificado = [x[0] for x in selected_indices]

In [ ]:
def processar_triclusters(selected_indices, df, partes_XYZ_path):

    with open(partes_XYZ_path, "r") as arquivo:
        partes_XYZ = json.load(arquivo)

    filtered_tctriclass = [tctriclass[idx] for idx in selected_indices]
    tctri_classes, tctri_ratios, tctri_elements = zip(*filtered_tctriclass)

    triclusters_selecionados = [{"nome": f"tctri{i}", "info": partes_XYZ[i]} for i in selected_indices]

    triselect = {}

    for i, tricluster in enumerate(triclusters_selecionados):
        columns_to_keep = ['X']
        nome_tricluster = tricluster['nome']
        coordenadas_tricluster = tricluster['info']

        for x, y, z in coordenadas_tricluster:
            column_name = f'y{y}z{z}'
            if column_name not in columns_to_keep:
                columns_to_keep.append(column_name)

        tctri_df = df[columns_to_keep].loc[df['X'].isin([f'x{x}' for x, _, _ in coordenadas_tricluster])]

        tctri_df = pd.merge(tctri_df, df[['X', ' class']], on='X', how='left')

        classe_majoritaria = tctri_df[' class'].mode().values[0]
        total_elementos = len(tctri_df)
        elementos_classe_majoritaria = len(tctri_df[tctri_df[' class'] == classe_majoritaria])
        razao_classe_majoritaria = elementos_classe_majoritaria / total_elementos

        tricluster_info = {'dataframe': tctri_df, 'ratio': razao_classe_majoritaria}
        triselect[f'{nome_tricluster}'] = tricluster_info

    return triselect

partes_XYZ_path = 'partes_XYZ.txt'
triselect1 = processar_triclusters(selected_indices_modificado, df, partes_XYZ_path)

In [ ]:
def remover_por_ratio_minimo(triselect, ratio_minimo):
    triselect_filtrado = {}
    for chave, valor in triselect.items():
        if valor['ratio'] >= ratio_minimo:
            triselect_filtrado[chave] = valor['dataframe']
    return triselect_filtrado


ratio_minimo = 0
triselect = remover_por_ratio_minimo(triselect1, ratio_minimo)

In [ ]:
import pickle

nome_do_arquivo_pickle = 'triselect.pickle'

with open(nome_do_arquivo_pickle, 'wb') as f:
    pickle.dump(triselect, f)


In [ ]:
numero_de_triclusters = len(triselect)
print("Number of triclusters:", numero_de_triclusters)

In [ ]:
import pandas as pd

def criar_dataset_especial(triselect, n_classes, n_scale):
    dataset_especial = []
    min_ratio = 1 / n_classes
    scale_steps = (1 - min_ratio) / n_scale
    
    intervalos_repeticoes_i = {}
    num_rep = []
    for i in range(n_scale):
        intervalo_inferior = min_ratio + i * scale_steps
        intervalo_superior = min_ratio + (i + 1) * scale_steps - 0.000000000000001 if i < n_scale - 1 else 1.0
        
        num_repeticoes = int(2*i-1) if i > 0 else 1
        intervalos_repeticoes_i[(intervalo_inferior, intervalo_superior)] = num_repeticoes
        num_rep.append(num_repeticoes)
    
    for key, df in triselect.items():
        intervalos_repeticoes = intervalos_repeticoes_i.copy()  
        
        num_elementos = len(df)
        
        if num_elementos <= 3:
            for j, intervalo in enumerate(intervalos_repeticoes):
                if j <= 3:
                    intervalos_repeticoes[intervalo] = num_rep[0]
                else:
                    if len(num_rep) >= 2:
                        intervalos_repeticoes[intervalo] = num_rep[1]
                    else:
                        intervalos_repeticoes[intervalo] = num_rep[0]
        elif 4 <= num_elementos <= 5:
            for j, intervalo in enumerate(intervalos_repeticoes):
                if j <= 1:
                    intervalos_repeticoes[intervalo] = num_rep[0]
                elif 1 < j <= 3:
                    if len(num_rep) >= 2:
                        intervalos_repeticoes[intervalo] = num_rep[1]
                    else:
                        intervalos_repeticoes[intervalo] = num_rep[0]
                else:
                    if len(num_rep) >= 3:
                        intervalos_repeticoes[intervalo] = num_rep[2]
                    elif len(num_rep) >= 2:
                        intervalos_repeticoes[intervalo] = num_rep[1]
                    else:
                        intervalos_repeticoes[intervalo] = num_rep[0]
        
        classe_maioritaria = df[' class'].mode().values
        if len(classe_maioritaria) == 1:
            ratio_classe_maioritaria = sum(df[' class'] == classe_maioritaria[0]) / len(df)
            
            for intervalo, num_repeticoes in intervalos_repeticoes.items():
                if intervalo[0] <= ratio_classe_maioritaria <= intervalo[1]:
                    colunas_especial = df.columns.tolist()
                    if 'X' in colunas_especial:
                        colunas_especial.remove('X')  
                    if ' class' in colunas_especial:
                        colunas_especial.remove(' class')  
                    colunas_especial.append(str(classe_maioritaria[0]))  
                    
                    for _ in range(num_repeticoes):
                        dataset_especial.append(colunas_especial)
                    break
    
    dataset_especial = pd.DataFrame({'colunas': dataset_especial})
    
    return dataset_especial

dataset_especial = criar_dataset_especial(triselect, n_classes, 5)
dataset_especial


In [ ]:
caminho_arquivo = 'dataset_especial.txt'

dataset_especial.to_csv(caminho_arquivo, index=False, sep='\t')


In [ ]:
from pyfpgrowth import find_frequent_patterns, generate_association_rules
transactions = dataset_especial['colunas'].tolist()
len(transactions)

In [ ]:
all_transactions=[transaction for transaction in transactions]

filename = "transactions_all.txt"
with open(filename, 'w') as f:
    for transaction in all_transactions:
        f.write(' '.join(transaction) + '\n')